# 준비

In [24]:
import numpy as np
import pandas as pd

In [25]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)
df.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']
df.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 0으로 채워진 넘파이 X 배열 생성
X = np.zeros(len(df)*(len(df.columns)-1)).reshape(len(df),len(df.columns)-1)

# X값에 데이터 적용
for i,col in enumerate(df.columns[1:]) :
    X[:,i] = df[col].values
# y값 설정
y = df['Class label'].values

X.shape, y.shape

((178, 13), (178,))

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify = y,
                                                    random_state = 1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((142, 13), (36, 13), (142,), (36,))

# LGBM

In [28]:
# LGBM
from lightgbm import LGBMClassifier

# wine 데이터셋은 데이터 양이 적어 LGBM이 적합하지는 않음.

model = LGBMClassifier(max_depth = 10, # 트리의 최대 깊이. 과적합을 다룰 때 사용.
                                       # feature가 많으면 크게 설정.
                       learning_rate = 0.01,
                       n_estimators  = 400,

                       min_child_samples = 20, # = min_data_in_leaf 과적합을 다룰 때 사용. 
                                               # 이 값을  크게 설정하면 트리가 깊어지는것을 방지
                       
                       num_leaves = 31,      # 트리의 leave 수 조절. 과적합을 다룰 때 사용.
                       
                       objective = 'multiclass', # 회귀문제인지 분류문제인지를 정함.
                                               # regression, binary, multiclass        
                       
                       boosting = 'gbdt',      # 실행하고자 하는 알고리즘 타입을 정의
                                               # gbdt, rf, dart, goss
                       
                       metric = 'multi_logloss', # 모델의 손실을 정함
                                                  # mae, mse, binary_logloss, multi_logloss
                      
                       scale_pos_weight = 1)      # 불균형 데이터셋에서 가중치를 증가시킴.
                                                  # 1이 디폴트값, 불균형 정도에 따라 조절   

# num_leaves의 개수를 중심으로 min_child_samples, max_depth를 
# 함께 조정하면서 모델의 복잡도를 줄이는 것이 기본 튜닝

In [29]:
# K-겹 교차검증
from sklearn.model_selection import cross_val_score
lgbm = model.fit(X_train,y_train)

scores = cross_val_score(estimator= lgbm,
                           X = X_train, y = y_train, cv = 10, 
                           scoring = 'accuracy')
print(f'K-겹 교차검증 Acc : {scores.mean() : .3f} (+/- {scores.std():.3f})')

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boostin

In [30]:
# 스코어
train_score = lgbm.score(X_train, y_train)
test_score = lgbm.score(X_test, y_test)
print(f'스코어 Acc : {train_score : .3f} {test_score : .3f}')

스코어 Acc :  1.000  1.000


# xgbm

* `핵심 파라미터`
    * General parameters - 어떤 모델을 부스팅할 것인가. (트리 or 선형)
    * Booster parameters - 어떤 부스팅을 쓸것인지.
    * Learning task parameters - 회귀/분류/랭킹 등에 따라 달라짐

* `General parameters`
    * booster - boost 타입
        - [gbtree*, gblinear, dart]
    * verbose - 출력메세지
        - 0 (silent), 1 (warning), 2 (info), 3 (debug)

* `Tree Boost parameters`
    * eta - 학습률
        - default = 0.3, [0~1] 
    * gamma - min-split-loss : gamma 가 커질수록 가중치 변동이 적음
        - default = 0, [0~무한)
    *  max_depth - 트리 최대 깊이
        - default = 6, [0~무한)
        - 0은 제한없음, Tree 모델에서는 항상 제한을 둬야함
    * min_child_weight - child 노드의 최소 가중치
        - default = 1, [0~무한)
        - 값이 커질수록 가중치 변동이 적음
    * max_delta_step - 최대 가중치 변동량
        - default = 0, [0~무한)
        - 잘 사용하지 않는 파라미터이지만, 로지스틱 회기에 1~10 사이 값을 주어 사용하면 효과가 있음
    * subsample - train instances 의 비율
        - default = 1, (0~1]
        - 비율로 추출하며 표본을 만들어서 fit 을 함
        - 과적합을 막음
    * lambda - 가중치의 L2 정규화 텀
        - default = 1
        - 값이 커지면 가중치 변동이 적음
    * alpha - 가중치의 L1 정규화 텀
        - default = 0
        - 값이 커지면 가중치 변동이 적음
    * max_leaves - 최대 노드 수
        - default = 0
        - tree method 가 exact 일때는 사용하지 않음
        
* `Linear Boost parameters`
    * lambda - default = 0
    * alpha - default = 0
    * feature_selector
        - default = cyclic, [cyclic, shuffle, random, greedy]

* `Learning Task parameters`
    * objective - 목적
        - default = reg:squarederror - 제곱오차 회귀
        - reg:logistic - 로지스틱 회귀
        - binary:logistic - 로지스틱 이진분류
        - binary:logitraw - 로지스틱 이진분류(회귀)
        - multi:softmax - 소프트맥스 다중 분류
        - multi:softprob - 소프트맥스와 같지만 output 이 matrix 임
    * eval_metric - 평가 지표
        - rmse : root mean square error
        - rmsle : root mean square log error
        - mae : mean absolute error
        - mape : mean absolute percentage error
        - mphe : mean Pseudo Huber error
        - logloss 
        - error : 분류 #(wrong cases)/#(all cases)
        - merror : 다중 분류 error
        - auc : 분류
        - logloss
        - mlogloss - 다중 클래스 logloss.
        - 직접제작한 함수 가능

In [31]:
from xgboost import XGBClassifier
y_train

array([3, 1, 3, 2, 3, 1, 2, 1, 2, 2, 2, 1, 3, 2, 1, 1, 1, 2, 3, 2, 3, 3,
       2, 3, 3, 2, 2, 1, 2, 2, 3, 3, 1, 2, 3, 2, 1, 2, 3, 3, 2, 2, 1, 1,
       1, 3, 2, 3, 1, 2, 2, 3, 1, 2, 2, 2, 3, 1, 1, 3, 2, 2, 3, 1, 1, 2,
       3, 1, 2, 1, 2, 3, 2, 2, 1, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3,
       2, 1, 2, 2, 3, 3, 2, 3, 1, 2, 2, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 1,
       1, 2, 3, 1, 2, 2, 1, 1, 1, 2, 2, 3, 2, 3, 3, 1, 1, 2, 1, 2, 3, 2,
       1, 1, 2, 2, 2, 3, 1, 3, 1, 3], dtype=int64)

In [32]:
y_train = y_train-1
y_train

array([2, 0, 2, 1, 2, 0, 1, 0, 1, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2, 1, 2, 2,
       1, 2, 2, 1, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0,
       0, 2, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1, 2, 0, 0, 2, 1, 1, 2, 0, 0, 1,
       2, 0, 1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       1, 0, 1, 1, 2, 2, 1, 2, 0, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 0,
       0, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 1, 0, 1, 2, 1,
       0, 0, 1, 1, 1, 2, 0, 2, 0, 2], dtype=int64)

In [33]:
xgbm = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [34]:
xgbm.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [39]:
# K-겹 교차검증
scores = cross_val_score(estimator= xgbm,
                           X = X_train, y = y_train, cv = 10, 
                           scoring = 'accuracy')
print(f'K-겹 교차검증 Acc : {scores.mean() : .3f} (+/- {scores.std():.3f})')

K-겹 교차검증 Acc :  0.972 (+/- 0.034)


In [41]:
# 스코어
train_score = xgbm.score(X_train, y_train)
test_score = xgbm.score(X_test, y_test-1)

print(f'스코어 Acc : {train_score : .3f} {test_score : .3f}')

스코어 Acc :  1.000  0.972


In [44]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0